In [1]:
import numpy as np
import pandas as pd
import time
import folium

from batch_classes import LocationType, Location, Dropoff, Pickup, Order

from preprocessor import preprocess_excel_to_objects, get_distance_matrix, parse_distance_matrix
from batch_manager import BatchManager

pd.set_option('display.max_rows', None)  # row limit off

In [2]:
# Google API - Distance Matrix API
YOUR_API_KEY = ""

In [3]:
def batch_optimization(
    sheet_name=None, 
    order_num=20, 
    max_capacity=None, 
    max_distance=None, 
    delay_minute_per_task=None, 
    first_order_pickup_duration=None
):
    
    start_time = time.time()
    
    orders, locations = preprocess_excel_to_objects(
        file_path='../../../repository/route_model/raw_data/Data-for-Batch-Optimization.xlsx', 
        sheet_name=sheet_name,
        order_num=order_num,        
    )

    # API_KEY required while getting distance&duration matrix by Google API (Distance Matrix API)
    matrix, response_list = get_distance_matrix(sheet_name, locations, YOUR_API_KEY)
    if isinstance(matrix, tuple):
        matrix = matrix[0]
    
    bm = BatchManager(
        sheet_name, orders, locations, matrix, 
        max_capacity=max_capacity,
        max_distance=max_distance,
        delay_minute_per_task=delay_minute_per_task,
        first_order_pickup_duration=first_order_pickup_duration,
    )
    
    while len(bm.order_pool) > 0:
        # Single batch process
        
        # 2. sort by urgency of time windows
        location_list = [[order.pickup, order.dropoff] for order in bm.order_pool]
        bm.location_pool = [location for sublist in location_list for location in sublist]
        bm.location_pool.sort()
        
        # 3. select most urgent location(pickup/dropoff)
        most_urgent_location = bm.find_most_urgent_location()
        target_order = next(
            order for order in orders
            if order.order_id == most_urgent_location.order_id
        )
        bm.add_order_to_batch(target_order)
    
        bm.batch_locations = bm.gather_locations_from_order(target_order)
        bm.optimize_route()
    
        while True:
            # 4. 그 주문으로부터 평균 거리가 가까운 애들을 순차적으로 하나씩 고름
            target_order = bm.find_close_order()
            
            # 4-1.후보 주문이 없을 경우 루프 종료
            if target_order is None:
                break

            bm.current_total_distance = bm.calculate_total_distance()
            
            # 5. 주문 경로 가능한지 확인하고 거리 최소화하기 (RouteOptimizer)
            bm.batch_locations = bm.gather_locations_from_order(target_order)
            succeed = bm.optimize_route()

            # 6. 성공 시 추가
            if succeed and bm.calculate_avg_distance_with_batch_orders(target_order) < bm.distance_threshold:
                bm.add_order_to_batch(target_order)
                bm.reset_failed_pool()
            
            # 7. 넘으면 추가 X
            else:
                bm.delete_location_from_order(target_order)
                bm.add_to_failed_pool(target_order)
                bm.update_arrival_time()
    
        # 8. 배치 마무리 (배치 객체 생성, order_pool과 locations에서 값 제거)
        bm.add_current_batch_to_list()
        bm.delete_batch_from_pool()
        bm.reset_failed_pool()
    
    print(f"Number of batch: {len(bm.batch_list)}")
    print(f"Time spent: {time.time()-start_time:.2f} s")

    return bm

In [4]:
bm_1 = batch_optimization(
    sheet_name="SameDay Orders(8 Hours SLA)", 
    order_num=20, 
    max_capacity=30, 
    max_distance=120,
    delay_minute_per_task=5,
    first_order_pickup_duration=10,
)

Number of batch: 7
Time spent: 0.39 s


In [5]:
bm_2 = batch_optimization(
    sheet_name="Instant Orders(1-4 Hours SLA)", 
    order_num=20, 
    max_capacity=30, 
    max_distance=120,
    delay_minute_per_task=5,
    first_order_pickup_duration=10,
)

Number of batch: 11
Time spent: 0.11 s


In [6]:
bm_3 = batch_optimization(
    sheet_name="Mixed Orders(SameDay+Instant)", 
    order_num=20, 
    max_capacity=30, 
    max_distance=120,
    delay_minute_per_task=5,
    first_order_pickup_duration=10,
)

Number of batch: 9
Time spent: 0.17 s


# Map

## Map - show all routes

In [7]:
# all route
bm_1.add_all_route_to_map(number_version=False)

## Map - 8 Hours

In [8]:
bm_1.add_single_route_to_map(batch_id=1, number_version=True)

## Map - 1~4 Hours

In [9]:
bm_2.add_single_route_to_map(batch_id=2, number_version=True)

## Map - Mixed

In [10]:
bm_3.add_single_route_to_map(batch_id=2, number_version=True)

# Table

## Table - 8 Hours

In [11]:
bm_1.get_batch_metrics_table(batch_no=1)

,ID,Type,Weight,Arrival,Deadline,(Created),Acc Dist,Acc Weight
0,23,PICKUP,8.12 kg,08:40:00,16:30:00,08:30:00,0.00 km,8.12 kg
1,4,PICKUP,8.7 kg,08:48:01,16:35:00,08:35:00,0.42 km,16.82 kg
2,40,PICKUP,3.52 kg,09:01:17,16:10:00,08:10:00,2.43 km,20.34 kg
3,12,PICKUP,8.57 kg,09:12:31,16:00:00,08:00:00,4.77 km,28.91 kg
4,12,DROPOFF,,09:35:45,16:00:00,08:00:00,12.55 km,20.34 kg
5,28,PICKUP,9.23 kg,09:58:39,16:35:00,08:35:00,19.10 km,29.57 kg
6,40,DROPOFF,,10:15:37,16:10:00,08:10:00,22.37 km,26.05 kg
7,15,PICKUP,2.17 kg,10:24:21,16:15:00,08:15:00,23.66 km,28.22 kg
8,28,DROPOFF,,10:30:43,16:35:00,08:35:00,24.18 km,18.99 kg
9,4,DROPOFF,,10:38:27,16:35:00,08:35:00,25.13 km,10.29 kg


## Table - 1~4 Hours

In [12]:
bm_2.get_batch_metrics_table(batch_no=2)

,ID,Type,Weight,Arrival,Deadline,(Created),Acc Dist,Acc Weight
0,1,PICKUP,7.54 kg,08:10:00,09:00:00,08:00:00,0.00 km,7.54 kg
1,28,PICKUP,9.23 kg,08:17:10,09:05:00,08:05:00,0.33 km,16.77 kg
2,28,DROPOFF,,08:30:10,09:05:00,08:05:00,2.50 km,7.54 kg
3,1,DROPOFF,,08:43:57,09:00:00,08:00:00,5.06 km,0.00 kg
4,25,PICKUP,3.29 kg,08:59:58,09:15:00,08:15:00,7.52 km,3.29 kg
5,25,DROPOFF,,09:14:44,09:15:00,08:15:00,11.00 km,0.00 kg
6,31,PICKUP,3.86 kg,09:23:16,10:00:00,08:00:00,11.99 km,3.86 kg
7,31,DROPOFF,,09:35:40,10:00:00,08:00:00,15.07 km,0.00 kg
8,37,PICKUP,3.25 kg,09:46:59,10:25:00,08:25:00,18.11 km,3.25 kg
9,37,DROPOFF,,10:00:25,10:25:00,08:25:00,22.04 km,0.00 kg


## Table - Mixed

In [13]:
bm_3.get_batch_metrics_table(batch_no=2)

,ID,Type,Weight,Arrival,Deadline,(Created),Acc Dist,Acc Weight
0,84,PICKUP,2.68 kg,08:35:00,16:25:00,08:25:00,0.00 km,2.68 kg
1,72,PICKUP,3.45 kg,08:45:59,16:30:00,08:30:00,1.10 km,6.13 kg
2,61,PICKUP,4.82 kg,08:52:52,16:00:00,08:00:00,1.39 km,10.95 kg
3,96,PICKUP,1.8 kg,08:59:12,16:30:00,08:30:00,1.66 km,12.75 kg
4,64,PICKUP,6.11 kg,09:05:49,16:15:00,08:15:00,2.03 km,18.86 kg
5,72,DROPOFF,,09:24:34,16:30:00,08:30:00,8.23 km,15.41 kg
6,84,DROPOFF,,09:33:11,16:25:00,08:25:00,9.42 km,12.73 kg
7,64,DROPOFF,,09:41:04,16:15:00,08:15:00,10.31 km,6.62 kg
8,61,DROPOFF,,09:50:43,16:00:00,08:00:00,12.36 km,1.80 kg
9,96,DROPOFF,,10:01:27,16:30:00,08:30:00,14.48 km,0.00 kg


# Stats

In [14]:
bm_1.optimize_stats()

Number of batch : 7

Total distance   : 122.94 km
Average distance : 17.56 km
Maximum distance : 74.50 km
Minimum distance : 6.05 km


Total duration   : 08:18:09
Average duration : 01:11:09
Maximum duration : 05:36:47
Minimum duration : 00:21:46


In [15]:
bm_2.optimize_stats()

Number of batch : 11

Total distance   : 131.50 km
Average distance : 11.95 km
Maximum distance : 22.04 km
Minimum distance : 4.88 km


Total duration   : 08:33:25
Average duration : 00:46:40
Maximum duration : 01:50:25
Minimum duration : 00:17:19


In [16]:
bm_3.optimize_stats()

Number of batch : 9

Total distance   : 107.46 km
Average distance : 11.94 km
Maximum distance : 29.79 km
Minimum distance : 6.50 km


Total duration   : 07:29:38
Average duration : 00:49:57
Maximum duration : 02:11:09
Minimum duration : 00:22:49
